### Installations

In [18]:
!pip --q install langchain huggingface_hub openai langchain-openai

### Imports

In [2]:
import os

The core idea of the library is that we can “chain” together different components to create more advanced use cases around LLMs. Chains may consist of multiple components from several modules:

1. **Prompt templates**: Prompt templates are templates for different types of prompts. Like “chatbot” style templates, ELI5 question-answering, etc
2. **LLMs**: Large language models like GPT-3, BLOOM, etc
3. **Agents**: Agents use LLMs to decide what actions should be taken. Tools like web search or calculators can be used, and all are packaged into a logical loop of operations.
4. **Memory**: Short-term memory, long-term memory.

# Hugging Face Hub

### Template

In [3]:
from langchain import PromptTemplate

template = """Question: {question}

Answer: """
prompt = PromptTemplate(
        template=template,
    input_variables=['question']
)

question = "What is the most famous song from Eurovision?"

/Users/sophiayeremeeva/Desktop/CS/ML/code/langchain/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### LLM Chain

In [4]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'hf'

In [5]:
from langchain import HuggingFaceHub, LLMChain

flan_t5 = HuggingFaceHub(repo_id='google/flan-t5-large', model_kwargs={'temperature':1e-10})

llm_chain = LLMChain(prompt=prompt, llm=flan_t5)

print(llm_chain.run(question))

/Users/sophiayeremeeva/Desktop/CS/ML/code/langchain/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.huggingface_hub.HuggingFaceHub` was deprecated in langchain-community 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/Users/sophiayeremeeva/Desktop/CS/ML/code/langchain/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


i want to be a star


### Multiple questions

In [6]:
qs = [
    {'question': "How many countries are in European Union?"},
    {'question': "How many official languages aer in Switzerland?"},
    {'question': "What is a rank of EPFL?"}
]

In [7]:
result = llm_chain.generate(qs)

In [8]:
print(result)

generations=[[Generation(text='71')], [Generation(text='ten')], [Generation(text='tenth')]] llm_output=None run=[RunInfo(run_id=UUID('f0cc1522-a9cc-492d-8c79-b3adf6845708')), RunInfo(run_id=UUID('b7fc633b-ba96-45fa-aa83-5578f82c5a7c')), RunInfo(run_id=UUID('cd0db1f5-867a-440b-b323-abccb5ec8e8e'))]


### Multiple questions in one prompt

In [9]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""

In [10]:
long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

In [11]:
llm_chain = LLMChain(prompt=long_prompt, llm=flan_t5)

In [12]:
qs_str = ("How many countries are in European Union?\n" +
          "How many official languages aer in Switzerland?\n" +
          "What is a rank of EPFL?")

In [13]:
print(llm_chain.run(qs_str))

The European Football League (EFL) is the governing body of football in the European Union


# OpenAI

In [19]:
from langchain_openai import ChatOpenAI

os.environ['OPENAI_API_KEY'] = 'sk'

gpt = ChatOpenAI(model_name='gpt-4-turbo-preview')

In [20]:
llm_chain = LLMChain(prompt=prompt, llm=gpt)

In [21]:
print(llm_chain.run(question))

The most famous song from the Eurovision Song Contest is arguably "Waterloo" by ABBA, which won the competition for Sweden in 1974. This victory launched ABBA's international career, making them one of the best-selling music artists of all time. The song "Waterloo" itself has become an iconic Eurovision anthem, celebrated for its catchy tune and memorable performance.


In [22]:
llm_chain.generate(qs)

LLMResult(generations=[[ChatGeneration(text='As of my last update in April 2023, there are 27 countries in the European Union (EU).', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='As of my last update in April 2023, there are 27 countries in the European Union (EU).', response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 20, 'total_tokens': 43}, 'model_name': 'gpt-4-turbo-preview', 'system_fingerprint': 'fp_f38f4d6482', 'finish_reason': 'stop', 'logprobs': None}, id='run-d2bdc00a-70b9-41d5-b5e0-7526e48e4776-0'))], [ChatGeneration(text='Switzerland officially recognizes four languages: German, French, Italian, and Romansh.', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Switzerland officially recognizes four languages: German, French, Italian, and Romansh.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 20, 'total_tokens': 37}, 'model_name': 'gpt-4-tu

In [23]:
print(llm_chain.run(qs_str))

As of my last update in April 2023, there are 27 countries in the European Union (EU). The United Kingdom was the latest country to leave the EU, officially departing on January 31, 2020, in an event commonly referred to as Brexit.

Switzerland has four official languages: German, French, Italian, and Romansh. These languages reflect the country's diverse linguistic and cultural regions.

The École Polytechnique Fédérale de Lausanne (EPFL) is consistently ranked as one of the world's top universities in various global university rankings. Its specific rank can vary by source and year. For example, in the QS World University Rankings, EPFL has often been placed within the top 20 globally and is highly regarded, particularly in fields such as Engineering, Technology, and Sciences. However, for the most precise and current ranking, it's best to consult the latest version of the ranking report you are interested in, as university rankings can fluctuate annually based on various factors.
